## 태그 데이터 삽입

In [ ]:
import csv
import sqlite3

In [ ]:
indexpath = 'dbv3-index.db'
corepath = 'dbv3-core.db'
dbpath = 'dbv3-service.db'
conn = sqlite3.connect(dbpath)
cur = conn.cursor()
cur.execute(f'''ATTACH DATABASE '{indexpath}' AS DBINDEX;''')
cur.execute(f'''ATTACH DATABASE '{corepath}' AS DBCORE;''')
conn.commit()

In [ ]:
tagpath = 'data/address_tags_210525.csv'

with open(tagpath, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        cur.execute('''SELECT id FROM DBINDEX.AddrID
                       WHERE DBINDEX.AddrID.addr = ?;''', (row['Address'],))
        addrid = cur.fetchone()[0]

        cur.execute('''INSERT OR IGNORE INTO AddrTagID (tag)
                       VALUES (?);''', (Row['Tag'],))
        conn.commit()
        cur.execute('''SELECT id FROM AddrTagID
                       WHERE AddrTagID.tag = ?;''', (row['Tag'],))
        tagid = cur.fetchone()[0]

        cur.execute('''INSERT OR IGNORE INTO AddrTag (addr, tag)
                       VALUES (?, ?);''', (addrid, tagid))
        conn.commit()

In [ ]:
conn.close()

## 주소 특징 추출

In [ ]:
import os
import csv
import sqlite3
import statistics
import collections

import numpy as np
import pandas as pd
from scipy.stats import moment

import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib notebook

In [ ]:
# Cache DataFrame
cachepath = 'cache_address_210525.pickle'
if os.path.exists(cachepath):
    df = pd.read_pickle(cachepath)
else:
    df = pd.DataFrame()
df

In [ ]:
# Connect DB
conn = sqlite3.connect(':memory:')
cur = conn.cursor()
cur.execute('''ATTACH DATABASE './dbv3-index.db' AS DBINDEX;''')
cur.execute('''ATTACH DATABASE './dbv3-core.db' AS DBCORE;''')
cur.execute('''ATTACH DATABASE './dbv3-util.db' AS DBUTIL;''')
cur.execute('''ATTACH DATABASE './dbv3-service.db' AS DBSERVICE;''')
conn.commit()

In [ ]:
# Enqueue all of tagged addresses
queue = collections.deque()
subqueue = collections.deque()

for row in cur.execute('''SELECT DISTINCT addr FROM DBSERVICE.AddrTag;'''):
    queue.append(row[0])
print(f'Ready the tagged queue: {len(queue)}')

In [ ]:
# Process feature for addresses
while len(queue) != 0 or len(subqueue) != 0:
    fill = False
    if len(queue) != 0:
        addr = queue.popleft()
        fill = True
    else:
        addr = subqueue.popleft()
    if 'Addr' in df.columns and addr in df['Addr'].values:
        continue
    metric = {'Addr': addr}

    # Tx
    cur.execute('''SELECT COUNT(DISTINCT DBUTIL.Edge.tx)
                   FROM DBUTIL.Edge
                   WHERE DBUTIL.Edge.src == ?
                      OR DBUTIL.Edge.dst == ?;''', (addr, addr))
    Tx = cur.fetchone()[0]
    metric['Tx'] = Tx
    
    # InTx
    cur.execute('''SELECT COUNT(DISTINCT DBUTIL.Edge.tx)
                   FROM DBUTIL.Edge
                   WHERE DBUTIL.Edge.src == ?''', (addr,))
    InTx = cur.fetchone()[0]
    metric['InTx'] = InTx
    
    # OutTx
    cur.execute('''SELECT COUNT(DISTINCT DBUTIL.Edge.tx)
                   FROM DBUTIL.Edge
                   WHERE DBUTIL.Edge.dst == ?;''', (addr,))
    OutTx = cur.fetchone()[0]
    metric['OutTx'] = OutTx
    
    # BTC
    cur.execute('''SELECT SUM(DBUTIL.Edge.btc)
                   FROM DBUTIL.Edge
                   WHERE DBUTIL.Edge.src == ?
                      OR DBUTIL.Edge.dst == ?;''', (addr, addr))
    BTC = cur.fetchone()[0]
    metric['BTC'] = BTC
    
    # InBTC series
    lInBTC = []
    for row in cur.execute('''SELECT DBUTIL.Edge.btc
                              FROM DBUTIL.Edge
                              WHERE DBUTIL.Edge.src == ?;''', (addr,)):
        lInBTC.append(row[0])
    if len(lInBTC) == 0:
        lInBTC.append(0)
    InBTCMin = min(lInBTC)
    InBTCMax = max(lInBTC)
    InBTCSum = sum(lInBTC)
    InBTCMedian = statistics.median(lInBTC)
    InBTCM1 = statistics.mean(lInBTC)
    InBTCM2 = moment(lInBTC, moment=2)
    InBTCM3 = moment(lInBTC, moment=3)
    InBTCM4 = moment(lInBTC, moment=4)
    metric['InBTCMin'] = InBTCMin
    metric['InBTCMax'] = InBTCMax
    metric['InBTCSum'] = InBTCSum
    metric['InBTCMedian'] = InBTCMedian
    metric['InBTCM1'] = InBTCM1
    metric['InBTCM2'] = InBTCM2
    metric['InBTCM3'] = InBTCM3
    metric['InBTCM4'] = InBTCM4

    # OutBTC series
    lOutBTC = []
    for row in cur.execute('''SELECT DBUTIL.Edge.btc
                              FROM DBUTIL.Edge
                              WHERE DBUTIL.Edge.dst == ?;''', (addr,)):
        lOutBTC.append(row[0])
    if len(lOutBTC) == 0:
        lOutBTC.append(0)
    OutBTCMin = min(lOutBTC)
    OutBTCMax = max(lOutBTC)
    OutBTCSum = sum(lOutBTC)
    OutBTCMedian = statistics.median(lOutBTC)
    OutBTCM1 = statistics.mean(lOutBTC)
    OutBTCM2 = moment(lOutBTC, moment=2)
    OutBTCM3 = moment(lOutBTC, moment=3)
    OutBTCM4 = moment(lOutBTC, moment=4)
    metric['OutBTCMin'] = OutBTCMin
    metric['OutBTCMax'] = OutBTCMax
    metric['OutBTCSum'] = OutBTCSum
    metric['OutBTCMedian'] = OutBTCMedian
    metric['OutBTCM1'] = OutBTCM1
    metric['OutBTCM2'] = OutBTCM2
    metric['OutBTCM3'] = OutBTCM3
    metric['OutBTCM4'] = OutBTCM4
    
    # Use
    cur.execute('''SELECT COUNT(DBUTIL.Edge.tx)
                   FROM DBUTIL.Edge
                   WHERE DBUTIL.Edge.src == ?
                      OR DBUTIL.Edge.dst == ?;''', (addr, addr))
    Use = cur.fetchone()[0]
    metric['Use'] = Use

    # InUse
    cur.execute('''SELECT COUNT(DBUTIL.Edge.tx)
                   FROM DBUTIL.Edge
                   WHERE DBUTIL.Edge.src == ?''', (addr,))
    InUse = cur.fetchone()[0]
    metric['InUse'] = InUse
    
    # OutUse
    cur.execute('''SELECT COUNT(DBUTIL.Edge.tx)
                   FROM DBUTIL.Edge
                   WHERE DBUTIL.Edge.dst == ?;''', (addr,))
    OutUse = cur.fetchone()[0]
    metric['OutUse'] = OutUse
    
    # Age series
    lAge = []
    for row in cur.execute('''SELECT DBCORE.BlkTime.unixtime
                              FROM DBCORE.BlkTime
                              INNER JOIN DBCORE.BlkTx ON DBCORE.BlkTx.blk = DBCORE.BlkTime.blk
                              WHERE DBCORE.BlkTx.tx IN
                              (SELECT DBCORE.TxIn.tx
                               FROM DBCORE.TxIn
                               INNER JOIN DBCORE.TxOut ON DBCORE.TxOut.tx = DBCORE.TxIn.ptx AND
                                                          DBCORE.TxOut.n = DBCORE.TxIn.pn
                               WHERE DBCORE.TxOut.addr = ?
                               UNION 
                               SELECT DISTINCT DBCORE.TxOut.tx
                               FROM DBCORE.TxOut
                               WHERE DBCORE.TxOut.addr = ?);''', (addr, addr)):
        lAge.append(row[0])
    if len(lAge) == 0:
        lAge.append(0)
    Age = max(lAge) - min(lAge)
    AgeMin = min(lAge)
    AgeMax = max(lAge)
    AgeM1 = statistics.mean(lAge)
    AgeM2 = moment(lAge, moment=2)
    AgeM3 = moment(lAge, moment=3)
    AgeM4 = moment(lAge, moment=4)
    metric['Age'] = Age
    metric['AgeMin'] = AgeMin
    metric['AgeMax'] = AgeMax
    metric['AgeM1'] = AgeM1
    metric['AgeM2'] = AgeM2
    metric['AgeM3'] = AgeM3
    metric['AgeM4'] = AgeM4
    
    # InAgemmmm1~m4s
    lInAge = []
    for row in cur.execute('''SELECT DBCORE.BlkTime.unixtime
                              FROM DBCORE.BlkTime
                              INNER JOIN DBCORE.BlkTx ON DBCORE.BlkTx.blk = DBCORE.BlkTime.blk
                              WHERE DBCORE.BlkTx.tx IN
                              (SELECT DBCORE.TxIn.tx
                               FROM DBCORE.TxIn
                               INNER JOIN DBCORE.TxOut ON DBCORE.TxOut.tx = DBCORE.TxIn.ptx AND
                                                          DBCORE.TxOut.n = DBCORE.TxIn.pn
                               WHERE DBCORE.TxOut.addr = ?);''', (addr,)):
        lInAge.append(row[0])
    if len(lInAge) == 0:
        lInAge.append(0)
    InAge = max(lInAge) - min(lInAge)
    InAgeMin = min(lInAge)
    InAgeMax = max(lInAge)
    InAgeM1 = statistics.mean(lInAge)
    InAgeM2 = moment(lInAge, moment=2)
    InAgeM3 = moment(lInAge, moment=3)
    InAgeM4 = moment(lInAge, moment=4)
    metric['InAge'] = InAge
    metric['InAgeMin'] = InAgeMin
    metric['InAgeMax'] = InAgeMax
    metric['InAgeM1'] = InAgeM1
    metric['InAgeM2'] = InAgeM2
    metric['InAgeM3'] = InAgeM3
    metric['InAgeM4'] = InAgeM4
    
    # OutAgemmmm1~m4s
    lOutAge = []
    for row in cur.execute('''SELECT DBCORE.BlkTime.unixtime
                              FROM DBCORE.BlkTime
                              INNER JOIN DBCORE.BlkTx ON DBCORE.BlkTx.blk = DBCORE.BlkTime.blk
                              WHERE DBCORE.BlkTx.tx IN
                              (SELECT DISTINCT DBCORE.TxOut.tx
                               FROM DBCORE.TxOut
                               WHERE DBCORE.TxOut.addr = ?);''', (addr,)):
        lOutAge.append(row[0])
    if len(lOutAge) == 0:
        lOutAge.append(0)
    OutAge = max(lOutAge) - min(lOutAge)
    OutAgeMin = min(lOutAge)
    OutAgeMax = max(lOutAge)
    OutAgeM1 = statistics.mean(lOutAge)
    OutAgeM2 = moment(lOutAge, moment=2)
    OutAgeM3 = moment(lOutAge, moment=3)
    OutAgeM4 = moment(lOutAge, moment=4)
    metric['OutAge'] = OutAge
    metric['OutAgeMin'] = OutAgeMin
    metric['OutAgeMax'] = OutAgeMax
    metric['OutAgeM1'] = OutAgeM1
    metric['OutAgeM2'] = OutAgeM2
    metric['OutAgeM3'] = OutAgeM3
    metric['OutAgeM4'] = OutAgeM4
    
    # isP2PKH
    # isP2SH
    # isBech32
    ## P2PKH which begin with the number 1, eg: 1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2.
    ## P2SH type starting with the number 3, eg: 3J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy.
    ## Bech32 type starting with bc1, eg: bc1qar0srrr7xfkvy5l643lydnw9re59gtzzwf5mdq.
    isP2PKH = 0
    isP2SH = 0
    isBech32 = 0
    cur.execute('''SELECT DBINDEX.AddrID.addr
                   FROM DBINDEX.AddrID
                   WHERE DBINDEX.AddrID.id = ?''', (addr,))
    addr_str = cur.fetchone()[0]
    if addr_str.startswith('1'):
        isP2PKH = 1
    elif addr_str.startswith('3'):
        isP2SH = 1
    elif addr_str.startswith('bc1'):
        isBech32 = 1
    metric['isP2PKH'] = isP2PKH
    metric['isP2SH'] = isP2SH
    metric['isBech32'] = isBech32

    df = df.append(metric, ignore_index=True)
    
    if fill:
        for row in cur.execute('''SELECT DBCORE.TxOut.addr
                                  FROM DBCORE.TxIn
                                  INNER JOIN DBCORE.TxOut ON DBCORE.TxIn.ptx = DBCORE.TxOut.tx
                                         AND DBCORE.TxIn.pn = DBCORE.TxOut.n
                                  WHERE DBCORE.txIn.tx IN (
                                   SELECT DBCORE.TxIn.tx
                                   FROM DBCORE.TxIn
                                   INNER JOIN DBCORE.TxOut ON DBCORE.TxIn.ptx = DBCORE.TxOut.tx
                                          AND DBCORE.TxIn.pn = DBCORE.TxOut.n
                                   WHERE DBCORE.TxOut.addr = ?)
                                  GROUP BY DBCORE.TxOut.addr;''', (addr,)):
            subqueue.append(row[0])
    print(f'Left: {len(queue)}, {len(subqueue)}', end='\r')
        
df = df[['Addr', 
         'Tx', 'InTx', 'OutTx',
         'BTC',
         'InBTCMin', 'InBTCMax', 'InBTCSum', 'InBTCMedian',
         'InBTCM1', 'InBTCM2', 'InBTCM3', 'InBTCM4',
         'OutBTCMin', 'OutBTCMax', 'OutBTCSum', 'OutBTCMedian',
         'OutBTCM1', 'OutBTCM2', 'OutBTCM3', 'OutBTCM4',
         'Use', 'InUse', 'OutUse',
         'Age', 'AgeMin', 'AgeMax',
         'AgeM1', 'AgeM2', 'AgeM3', 'AgeM4',
         'InAge', 'InAgeMin', 'InAgeMax',
         'InAgeM1', 'InAgeM2', 'InAgeM3', 'InAgeM4',
         'OutAge', 'OutAgeMin', 'OutAgeMax',
         'OutAgeM1', 'OutAgeM2', 'OutAgeM3', 'OutAgeM4', 
         'isBech32', 'isP2PKH', 'isP2SH']]
df

## 거래 특징 추출

In [ ]:
# Enqueue all of tagged txes
queue = collections.deque()
subqueue = collections.deque()

for row in cur.execute('''SELECT DISTINCT tx FROM DBSERVICE.TxTag;'''):
    queue.append(row[0])
print(f'Ready the tagged queue: {len(queue)}')

In [ ]:
while len(queue) != 0:
    tx = queue.popleft()

    for row in cur.execute('''SELECT DBCORE.TxOut.addr
                            FROM DBCORE.TxIn
                            INNER JOIN DBCORE.TxOut ON DBCORE.TxIn.ptx = DBCORE.TxOut.tx
                                   AND DBCORE.TxIn.pn = DBCORE.TxOut.n
                            WHERE DBCORE.TxIn.tx = ?;''', (tx,)):
        subqueue.append(row[0])
    for row in cur.execute('''SELECT DBCORE.TxOut.addr
                              FROM DBCORE.TxOut
                              WHERE DBCORE.TxOut.tx = ?;''', (tx,)):
        subqueue.append(row[0])
print(f'Ready the tagged queue: {len(subqueue)}')

In [ ]:
# Process feature for addresses
while len(subqueue) != 0:
    addr = subqueue.popleft()
    if 'Addr' in df.columns and addr in df['Addr'].values:
        continue
    metric = {'Addr': addr}

    # Tx
    cur.execute('''SELECT COUNT(DISTINCT DBUTIL.Edge.tx)
                   FROM DBUTIL.Edge
                   WHERE DBUTIL.Edge.src == ?
                      OR DBUTIL.Edge.dst == ?;''', (addr, addr))
    Tx = cur.fetchone()[0]
    metric['Tx'] = Tx
    
    # InTx
    cur.execute('''SELECT COUNT(DISTINCT DBUTIL.Edge.tx)
                   FROM DBUTIL.Edge
                   WHERE DBUTIL.Edge.src == ?''', (addr,))
    InTx = cur.fetchone()[0]
    metric['InTx'] = InTx
    
    # OutTx
    cur.execute('''SELECT COUNT(DISTINCT DBUTIL.Edge.tx)
                   FROM DBUTIL.Edge
                   WHERE DBUTIL.Edge.dst == ?;''', (addr,))
    OutTx = cur.fetchone()[0]
    metric['OutTx'] = OutTx
    
    # BTC
    cur.execute('''SELECT SUM(DBUTIL.Edge.btc)
                   FROM DBUTIL.Edge
                   WHERE DBUTIL.Edge.src == ?
                      OR DBUTIL.Edge.dst == ?;''', (addr, addr))
    BTC = cur.fetchone()[0]
    metric['BTC'] = BTC
    
    # InBTC series
    lInBTC = []
    for row in cur.execute('''SELECT DBUTIL.Edge.btc
                              FROM DBUTIL.Edge
                              WHERE DBUTIL.Edge.src == ?;''', (addr,)):
        lInBTC.append(row[0])
    if len(lInBTC) == 0:
        lInBTC.append(0)
    InBTCMin = min(lInBTC)
    InBTCMax = max(lInBTC)
    InBTCSum = sum(lInBTC)
    InBTCMedian = statistics.median(lInBTC)
    InBTCM1 = statistics.mean(lInBTC)
    InBTCM2 = moment(lInBTC, moment=2)
    InBTCM3 = moment(lInBTC, moment=3)
    InBTCM4 = moment(lInBTC, moment=4)
    metric['InBTCMin'] = InBTCMin
    metric['InBTCMax'] = InBTCMax
    metric['InBTCSum'] = InBTCSum
    metric['InBTCMedian'] = InBTCMedian
    metric['InBTCM1'] = InBTCM1
    metric['InBTCM2'] = InBTCM2
    metric['InBTCM3'] = InBTCM3
    metric['InBTCM4'] = InBTCM4

    # OutBTC series
    lOutBTC = []
    for row in cur.execute('''SELECT DBUTIL.Edge.btc
                              FROM DBUTIL.Edge
                              WHERE DBUTIL.Edge.dst == ?;''', (addr,)):
        lOutBTC.append(row[0])
    if len(lOutBTC) == 0:
        lOutBTC.append(0)
    OutBTCMin = min(lOutBTC)
    OutBTCMax = max(lOutBTC)
    OutBTCSum = sum(lOutBTC)
    OutBTCMedian = statistics.median(lOutBTC)
    OutBTCM1 = statistics.mean(lOutBTC)
    OutBTCM2 = moment(lOutBTC, moment=2)
    OutBTCM3 = moment(lOutBTC, moment=3)
    OutBTCM4 = moment(lOutBTC, moment=4)
    metric['OutBTCMin'] = OutBTCMin
    metric['OutBTCMax'] = OutBTCMax
    metric['OutBTCSum'] = OutBTCSum
    metric['OutBTCMedian'] = OutBTCMedian
    metric['OutBTCM1'] = OutBTCM1
    metric['OutBTCM2'] = OutBTCM2
    metric['OutBTCM3'] = OutBTCM3
    metric['OutBTCM4'] = OutBTCM4
    
    # Use
    cur.execute('''SELECT COUNT(DBUTIL.Edge.tx)
                   FROM DBUTIL.Edge
                   WHERE DBUTIL.Edge.src == ?
                      OR DBUTIL.Edge.dst == ?;''', (addr, addr))
    Use = cur.fetchone()[0]
    metric['Use'] = Use

    # InUse
    cur.execute('''SELECT COUNT(DBUTIL.Edge.tx)
                   FROM DBUTIL.Edge
                   WHERE DBUTIL.Edge.src == ?''', (addr,))
    InUse = cur.fetchone()[0]
    metric['InUse'] = InUse
    
    # OutUse
    cur.execute('''SELECT COUNT(DBUTIL.Edge.tx)
                   FROM DBUTIL.Edge
                   WHERE DBUTIL.Edge.dst == ?;''', (addr,))
    OutUse = cur.fetchone()[0]
    metric['OutUse'] = OutUse
    
    # Age series
    lAge = []
    for row in cur.execute('''SELECT DBCORE.BlkTime.unixtime
                              FROM DBCORE.BlkTime
                              INNER JOIN DBCORE.BlkTx ON DBCORE.BlkTx.blk = DBCORE.BlkTime.blk
                              WHERE DBCORE.BlkTx.tx IN
                              (SELECT DBCORE.TxIn.tx
                               FROM DBCORE.TxIn
                               INNER JOIN DBCORE.TxOut ON DBCORE.TxOut.tx = DBCORE.TxIn.ptx AND
                                                          DBCORE.TxOut.n = DBCORE.TxIn.pn
                               WHERE DBCORE.TxOut.addr = ?
                               UNION 
                               SELECT DISTINCT DBCORE.TxOut.tx
                               FROM DBCORE.TxOut
                               WHERE DBCORE.TxOut.addr = ?);''', (addr, addr)):
        lAge.append(row[0])
    if len(lAge) == 0:
        lAge.append(0)
    Age = max(lAge) - min(lAge)
    AgeMin = min(lAge)
    AgeMax = max(lAge)
    AgeM1 = statistics.mean(lAge)
    AgeM2 = moment(lAge, moment=2)
    AgeM3 = moment(lAge, moment=3)
    AgeM4 = moment(lAge, moment=4)
    metric['Age'] = Age
    metric['AgeMin'] = AgeMin
    metric['AgeMax'] = AgeMax
    metric['AgeM1'] = AgeM1
    metric['AgeM2'] = AgeM2
    metric['AgeM3'] = AgeM3
    metric['AgeM4'] = AgeM4
    
    # InAgemmmm1~m4s
    lInAge = []
    for row in cur.execute('''SELECT DBCORE.BlkTime.unixtime
                              FROM DBCORE.BlkTime
                              INNER JOIN DBCORE.BlkTx ON DBCORE.BlkTx.blk = DBCORE.BlkTime.blk
                              WHERE DBCORE.BlkTx.tx IN
                              (SELECT DBCORE.TxIn.tx
                               FROM DBCORE.TxIn
                               INNER JOIN DBCORE.TxOut ON DBCORE.TxOut.tx = DBCORE.TxIn.ptx AND
                                                          DBCORE.TxOut.n = DBCORE.TxIn.pn
                               WHERE DBCORE.TxOut.addr = ?);''', (addr,)):
        lInAge.append(row[0])
    if len(lInAge) == 0:
        lInAge.append(0)
    InAge = max(lInAge) - min(lInAge)
    InAgeMin = min(lInAge)
    InAgeMax = max(lInAge)
    InAgeM1 = statistics.mean(lInAge)
    InAgeM2 = moment(lInAge, moment=2)
    InAgeM3 = moment(lInAge, moment=3)
    InAgeM4 = moment(lInAge, moment=4)
    metric['InAge'] = InAge
    metric['InAgeMin'] = InAgeMin
    metric['InAgeMax'] = InAgeMax
    metric['InAgeM1'] = InAgeM1
    metric['InAgeM2'] = InAgeM2
    metric['InAgeM3'] = InAgeM3
    metric['InAgeM4'] = InAgeM4
    
    # OutAgemmmm1~m4s
    lOutAge = []
    for row in cur.execute('''SELECT DBCORE.BlkTime.unixtime
                              FROM DBCORE.BlkTime
                              INNER JOIN DBCORE.BlkTx ON DBCORE.BlkTx.blk = DBCORE.BlkTime.blk
                              WHERE DBCORE.BlkTx.tx IN
                              (SELECT DISTINCT DBCORE.TxOut.tx
                               FROM DBCORE.TxOut
                               WHERE DBCORE.TxOut.addr = ?);''', (addr,)):
        lOutAge.append(row[0])
    if len(lOutAge) == 0:
        lOutAge.append(0)
    OutAge = max(lOutAge) - min(lOutAge)
    OutAgeMin = min(lOutAge)
    OutAgeMax = max(lOutAge)
    OutAgeM1 = statistics.mean(lOutAge)
    OutAgeM2 = moment(lOutAge, moment=2)
    OutAgeM3 = moment(lOutAge, moment=3)
    OutAgeM4 = moment(lOutAge, moment=4)
    metric['OutAge'] = OutAge
    metric['OutAgeMin'] = OutAgeMin
    metric['OutAgeMax'] = OutAgeMax
    metric['OutAgeM1'] = OutAgeM1
    metric['OutAgeM2'] = OutAgeM2
    metric['OutAgeM3'] = OutAgeM3
    metric['OutAgeM4'] = OutAgeM4
    
    # isP2PKH
    # isP2SH
    # isBech32
    ## P2PKH which begin with the number 1, eg: 1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2.
    ## P2SH type starting with the number 3, eg: 3J98t1WpEZ73CNmQviecrnyiWrnqRhWNLy.
    ## Bech32 type starting with bc1, eg: bc1qar0srrr7xfkvy5l643lydnw9re59gtzzwf5mdq.
    isP2PKH = 0
    isP2SH = 0
    isBech32 = 0
    cur.execute('''SELECT DBINDEX.AddrID.addr
                   FROM DBINDEX.AddrID
                   WHERE DBINDEX.AddrID.id = ?''', (addr,))
    addr_str = cur.fetchone()[0]
    if addr_str.startswith('1'):
        isP2PKH = 1
    elif addr_str.startswith('3'):
        isP2SH = 1
    elif addr_str.startswith('bc1'):
        isBech32 = 1
    metric['isP2PKH'] = isP2PKH
    metric['isP2SH'] = isP2SH
    metric['isBech32'] = isBech32

    df = df.append(metric, ignore_index=True)
    
    print(f'Left: {len(subqueue)}', end='\r')
        
df = df[['Addr', 
         'Tx', 'InTx', 'OutTx',
         'BTC',
         'InBTCMin', 'InBTCMax', 'InBTCSum', 'InBTCMedian',
         'InBTCM1', 'InBTCM2', 'InBTCM3', 'InBTCM4',
         'OutBTCMin', 'OutBTCMax', 'OutBTCSum', 'OutBTCMedian',
         'OutBTCM1', 'OutBTCM2', 'OutBTCM3', 'OutBTCM4',
         'Use', 'InUse', 'OutUse',
         'Age', 'AgeMin', 'AgeMax',
         'AgeM1', 'AgeM2', 'AgeM3', 'AgeM4',
         'InAge', 'InAgeMin', 'InAgeMax',
         'InAgeM1', 'InAgeM2', 'InAgeM3', 'InAgeM4',
         'OutAge', 'OutAgeMin', 'OutAgeMax',
         'OutAgeM1', 'OutAgeM2', 'OutAgeM3', 'OutAgeM4', 
         'isBech32', 'isP2PKH', 'isP2SH']]
df

In [ ]:
df.to_pickle(cachepath)
conn.close()

## 주소 태그 특징 추출

In [ ]:
tag_map = {'CHILD ABUSE MATERIAL': 'OTHER',
           'DARKNET MARKET': 'DARKNET MARKET',
           'EXCHANGE': 'EXCHANGE',
           'GAMBLING': 'GAMBLING',
           'HACK': 'OTHER',
           'HIGH RISK EXCHANGE': 'OTHER',
           'HOSTED WALLET': 'OTHER',
           'MERCHANT SERVICES': 'OTHER',
           'MINING POOL': 'MINING POOL',
           'MIXING': 'OTHER',
           'OTHER': 'OTHER',
           'P2P EXCHANGE': 'OTHER',
           'RANSOMWARE': 'RANSOMWARE',
           'SANCTIONS': 'OTHER',
           'SCAM': 'SCAM',
           'STOLEN FUNDS': 'OTHER',
           'TERRORIST FINANCING': 'OTHER',
          }

In [ ]:
ndf = pd.DataFrame()
ndf

In [ ]:
for key, value in tag_map.items():
    addrs = []
    for row in cur.execute('''SELECT DBSERVICE.AddrTag.addr
                              FROM DBSERVICE.AddrTag
                              INNER JOIN DBSERVICE.AddrTagID ON DBSERVICE.AddrTag.tag = DBSERVICE.AddrTagID.id
                              WHERE DBSERVICE.AddrTagID.tag = ?;''', (key,)):
        addrs.append(row[0])
    for addr in addrs:
        metric = {'Tag': value, 'Addr': addr}
        metric.update(df.loc[df['Addr']==addr,
                             ['Tx', 'InTx', 'OutTx',
                              'BTC',
                              'InBTCMin', 'InBTCMax', 'InBTCSum', 'InBTCMedian',
                              'InBTCM1', 'InBTCM2', 'InBTCM3', 'InBTCM4',
                              'OutBTCMin', 'OutBTCMax', 'OutBTCSum', 'OutBTCMedian',
                              'OutBTCM1', 'OutBTCM2', 'OutBTCM3', 'OutBTCM4',
                              'Use', 'InUse', 'OutUse',
                              'Age', 'AgeMin', 'AgeMax',
                              'AgeM1', 'AgeM2', 'AgeM3', 'AgeM4',
                              'InAge', 'InAgeMin', 'InAgeMax',
                              'InAgeM1', 'InAgeM2', 'InAgeM3', 'InAgeM4',
                              'OutAge', 'OutAgeMin', 'OutAgeMax',
                              'OutAgeM1', 'OutAgeM2', 'OutAgeM3', 'OutAgeM4', 
                              'isBech32', 'isP2PKH', 'isP2SH']].to_dict('records')[0])
        miaddrs = set()
        for row in cur.execute('''SELECT DBCORE.TxOut.addr
                                  FROM DBCORE.TxIn
                                  INNER JOIN DBCORE.TxOut ON DBCORE.TxIn.ptx = DBCORE.TxOut.tx
                                         AND DBCORE.TxIn.pn = DBCORE.TxOut.n
                                  WHERE DBCORE.txIn.tx IN (
                                   SELECT DBCORE.TxIn.tx
                                   FROM DBCORE.TxIn
                                   INNER JOIN DBCORE.TxOut ON DBCORE.TxIn.ptx = DBCORE.TxOut.tx
                                          AND DBCORE.TxIn.pn = DBCORE.TxOut.n
                                   WHERE DBCORE.TxOut.addr = ?)
                                  GROUP BY DBCORE.TxOut.addr;''', (addr,)):
            miaddrs.add(row[0])
        if len(miaddrs) > 0:
            miaddrs.remove(addr)
        if len(miaddrs) > 0:
            for feature in ['Tx', 'InTx', 'OutTx',
                            'BTC',
                            'InBTCMin', 'InBTCMax', 'InBTCSum', 'InBTCMedian',
                            'InBTCM1', 'InBTCM2', 'InBTCM3', 'InBTCM4',
                            'OutBTCMin', 'OutBTCMax', 'OutBTCSum', 'OutBTCMedian',
                            'OutBTCM1', 'OutBTCM2', 'OutBTCM3', 'OutBTCM4',
                            'Use', 'InUse', 'OutUse',
                            'Age', 'AgeMin', 'AgeMax',
                            'AgeM1', 'AgeM2', 'AgeM3', 'AgeM4',
                            'InAge', 'InAgeMin', 'InAgeMax',
                            'InAgeM1', 'InAgeM2', 'InAgeM3', 'InAgeM4',
                            'OutAge', 'OutAgeMin', 'OutAgeMax',
                            'OutAgeM1', 'OutAgeM2', 'OutAgeM3', 'OutAgeM4',]:
                metric[f'MI{feature}Sum'] = sum(df.loc[df['Addr'].isin(miaddrs), feature])
                metric[f'MI{feature}Min'] = min(df.loc[df['Addr'].isin(miaddrs), feature])
                metric[f'MI{feature}Max'] = max(df.loc[df['Addr'].isin(miaddrs), feature])
                metric[f'MI{feature}Median'] = statistics.median(df.loc[df['Addr'].isin(miaddrs), feature])
                metric[f'MI{feature}M1'] = statistics.mean(df.loc[df['Addr'].isin(miaddrs), feature])
                metric[f'MI{feature}M2'] = moment(df.loc[df['Addr'].isin(miaddrs), feature], moment=2)
                metric[f'MI{feature}M3'] = moment(df.loc[df['Addr'].isin(miaddrs), feature], moment=3)
                metric[f'MI{feature}M4'] = moment(df.loc[df['Addr'].isin(miaddrs), feature], moment=4)
            total = np.sum(df.loc[df['Addr'].isin(miaddrs), ['isBech32', 'isP2PKH', 'isP2SH']].values)
            for feature in ['isBech32', 'isP2PKH', 'isP2SH']:
                metric[f'MI{feature}'] = sum(df.loc[df['Addr'].isin(miaddrs), feature]) / total
        ndf = ndf.append(metric, ignore_index=True)
ndf

In [ ]:
feature_addr_path = 'feature_addr.pkl'
ndf.to_pickle(feature_addr_path)

In [ ]:
tag_list = ['LICIT', 'ILLICIT']

In [ ]:
tx_list = []
for tag in tag_list:
    for row in cur.execute('''SELECT DISTINCT tx 
                              FROM DBSERVICE.TxTag
                              INNER JOIN DBSERVICE.TxTagID ON DBSERVICE.TxTag.tag = DBSERVICE.TxTagID.id
                              WHERE DBSERVICE.TxTagID.tag = ?;''', (tag,)):
        tx_list.append((tag, row[0]))
print(len(tx_list))

In [ ]:
count = 0
for tag, tx in tx_list:
    inaddrs = []
    for row in cur.execute('''SELECT DBCORE.TxOut.addr
                              FROM DBCORE.TxIn
                              INNER JOIN DBCORE.TxOut ON DBCORE.TxIn.ptx = DBCORE.TxOut.tx
                                     AND DBCORE.TxIn.pn = DBCORE.TxOut.n
                              WHERE DBCORE.TxIn.tx = ?;''', (tx,)):
        inaddrs.append(row[0])
    outaddrs = []
    for row in cur.execute('''SELECT DBCORE.TxOut.addr
                              FROM DBCORE.TxOut
                              WHERE DBCORE.TxOut.tx = ?;''', (tx,)):
        outaddrs.append(row[0])
    addrs = inaddrs + outaddrs
    metric = {'Tag': tag, 'Tx': tx}
    for prefix, data in [('Addr', addrs),
                         ('InAddr', inaddrs),
                         ('OutAddr', outaddrs)]:
        metric[prefix] = sum(data)
        for feature in ['Tx', 'InTx', 'OutTx',
                        'BTC',
                        'InBTCMin', 'InBTCMax', 'InBTCSum', 'InBTCMedian',
                        'InBTCM1', 'InBTCM2', 'InBTCM3', 'InBTCM4',
                        'OutBTCMin', 'OutBTCMax', 'OutBTCSum', 'OutBTCMedian',
                        'OutBTCM1', 'OutBTCM2', 'OutBTCM3', 'OutBTCM4',
                        'Use', 'InUse', 'OutUse',
                        'Age', 'AgeMin', 'AgeMax',
                        'AgeM1', 'AgeM2', 'AgeM3', 'AgeM4',
                        'InAge', 'InAgeMin', 'InAgeMax',
                        'InAgeM1', 'InAgeM2', 'InAgeM3', 'InAgeM4',
                        'OutAge', 'OutAgeMin', 'OutAgeMax',
                        'OutAgeM1', 'OutAgeM2', 'OutAgeM3', 'OutAgeM4',]:
            metric[f'{prefix}{feature}Sum'] = sum(df.loc[df['Addr'].isin(data), feature])
            metric[f'{prefix}{feature}Min'] = min(df.loc[df['Addr'].isin(data), feature])
            metric[f'{prefix}{feature}Max'] = max(df.loc[df['Addr'].isin(data), feature])
            metric[f'{prefix}{feature}Median'] = statistics.median(df.loc[df['Addr'].isin(data), feature])
            metric[f'{prefix}{feature}M1'] = statistics.mean(df.loc[df['Addr'].isin(data), feature])
            metric[f'{prefix}{feature}M2'] = moment(df.loc[df['Addr'].isin(data), feature], moment=2)
            metric[f'{prefix}{feature}M3'] = moment(df.loc[df['Addr'].isin(data), feature], moment=3)
            metric[f'{prefix}{feature}M4'] = moment(df.loc[df['Addr'].isin(data), feature], moment=4)
            total = np.sum(df.loc[df['Addr'].isin(data), ['isBech32', 'isP2PKH', 'isP2SH']].values)
        for feature in ['isBech32', 'isP2PKH', 'isP2SH']:
            metric[f'{prefix}{feature}'] = sum(df.loc[df['Addr'].isin(data), feature]) / total
    ndf = ndf.append(metric, ignore_index=True)
    count = count + 1
    print(f'{count} / {len(tx_list)}')
ndf